# 04a - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

This notebook shows training a model directly within the runtime of the notebook environment.  Then the model is saved and moved to GCS for deployment to a Vertex AI Endpoint for online predictions.  The model training is done with [Tensorflow](https://www.tensorflow.org/), specifically [Keras](https://keras.io/), and was designed to show a neural network approach to logistic regression.  The training data batches are read from BigQuery using [Tensorflow I/O](https://www.tensorflow.org/io).

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Use Python Client for BigQuery
   -  Read the tables schema from BigQuery INFORMATION_SCHEMA
   -  Prepare the feature information for Tensorflow
-  Define a function that remaps the input data into features and target variables where target is one-hot encoded (classification model with 10 classes)
-  Set Tensorflow I/O read session
-  Demonstrate reading a single batch
-  Train a Tensorflow model
   -  Define the model layers
   -  Compile the model
   -  Fit the model
   -  Evaluate the model (loss, accuracy)
   -  Create prediction with the model
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict
-  Online Predictions with:
   -  REST call
   -  gcloud CLI

**Resources:**

-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-04.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '04a'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

# Model Training
BATCH_SIZE = 30

packages:

In [2]:
from google.cloud import bigquery

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

2021-09-17 15:34:20.059328: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [5]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get The Schema of The Training Data

In [6]:
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

In [7]:
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,statmike-mlops,digits,digits_prepped,p0,1,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
1,statmike-mlops,digits,digits_prepped,p1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
2,statmike-mlops,digits,digits_prepped,p2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
3,statmike-mlops,digits,digits_prepped,p3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
4,statmike-mlops,digits,digits_prepped,p4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,63,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
63,statmike-mlops,digits,digits_prepped,p63,64,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
64,statmike-mlops,digits,digits_prepped,target,65,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
65,statmike-mlops,digits,digits_prepped,target_OE,66,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN


## Prepare Inputs For Tensorflow Training

Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [8]:
OMIT = ['target_OE','splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

# all the columns in this data source are either float64 or int64
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into:
- features
- `target` - and one_hot encoded for multi-class classification

In [9]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

## Use Tensorflow I/O to Read Batches from BigQuery

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [10]:
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

In [11]:
train = bq_reader('TRAIN').parallel_read_rows().map(transTable).shuffle(BATCH_SIZE*3).batch(BATCH_SIZE)
validate = bq_reader('VALIDATE').parallel_read_rows().map(transTable).batch(BATCH_SIZE)
test = bq_reader('TEST').parallel_read_rows().map(transTable).batch(BATCH_SIZE)

2021-09-17 15:34:47.412926: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2299995000 Hz
2021-09-17 15:34:47.413533: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561e45431ce0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-17 15:34:47.413558: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-17 15:34:47.413736: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Review a single batch of the train data:

In [12]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns:\n',columns)
    print('\ntarget:\n',b)

columns:
 ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']

target:
 tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [

---
## Train the Model In The Notebook

Define the Model:

In [13]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(
    inputs = [v for v in feature_layer_inputs.values()],
    outputs = tf.keras.layers.Dense(10, activation = tf.nn.softmax)(feature_layer_outputs)
)
model.compile(
    optimizer = 'sgd',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
#tf.keras.utils.plot_model(model, show_shapes = True, show_dtype = True)

In [14]:
#model.summary()

Fit the Model:

In [15]:
history = model.fit(train, epochs = 25, validation_data = validate)

Epoch 1/25
49/49 [==============================] - 3s 58ms/step - loss: 3.6293 - accuracy: 0.5455 - val_loss: 1.2233 - val_accuracy: 0.7111
Epoch 2/25
49/49 [==============================] - 2s 39ms/step - loss: 0.8325 - accuracy: 0.8189 - val_loss: 0.3958 - val_accuracy: 0.8722
Epoch 3/25
49/49 [==============================] - 2s 41ms/step - loss: 0.4308 - accuracy: 0.8767 - val_loss: 0.2794 - val_accuracy: 0.9000
Epoch 4/25
49/49 [==============================] - 2s 36ms/step - loss: 0.3043 - accuracy: 0.9098 - val_loss: 0.2083 - val_accuracy: 0.9333
Epoch 5/25
49/49 [==============================] - 2s 33ms/step - loss: 0.2627 - accuracy: 0.9229 - val_loss: 0.2127 - val_accuracy: 0.9222
Epoch 6/25
49/49 [==============================] - 2s 33ms/step - loss: 0.1943 - accuracy: 0.9408 - val_loss: 0.2583 - val_accuracy: 0.9167
Epoch 7/25
49/49 [==============================] - 2s 41ms/step - loss: 0.1888 - accuracy: 0.9360 - val_loss: 0.2560 - val_accuracy: 0.9278
Epoch 8/25
49

In [16]:
history.history['loss'][-1]

0.05446118116378784

Evaluate the model with the test data:

In [17]:
loss, accuracy = model.evaluate(test)

6/6 [==============================] - 0s 12ms/step - loss: 0.2844 - accuracy: 0.8970


In [18]:
loss, accuracy = model.evaluate(validate)

6/6 [==============================] - 0s 11ms/step - loss: 0.2437 - accuracy: 0.9111


In [19]:
loss, accuracy = model.evaluate(train)

49/49 [==============================] - 1s 19ms/step - loss: 0.1615 - accuracy: 0.9463


Create Prediction from a batch of the test data and review first row:

In [20]:
model.predict(test.take(1))[0]

array([9.9042028e-01, 3.3717097e-08, 2.3498742e-06, 1.9295536e-10,
       1.4125196e-03, 2.0952461e-06, 8.1462283e-03, 1.0794516e-05,
       3.4893023e-06, 2.2126453e-06], dtype=float32)

---
## Serving

### Save The Model

In [20]:
model.save(URI)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-09-16 11:22:28.393108: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/models/04a/assets
INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/models/04a/assets


### Upload The Model

In [21]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/1431784041689907200/operations/3163579451745239040
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/1431784041689907200
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/1431784041689907200')


In [22]:
model.display_name

'04a_digits_20210916111550'

### Create An Endpoint

In [23]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/3724740774626066432/operations/5426638264498913280
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/3724740774626066432
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/3724740774626066432')


In [24]:
endpoint.display_name

'04a_digits_20210916111550'

### Deploy Model To Endpoint

In [25]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/1431784041689907200 to Endpoint : projects/691911073727/locations/us-central1/endpoints/3724740774626066432
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/3724740774626066432/operations/5250997879031463936
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/3724740774626066432


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [7]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME} LIMIT 10").to_dataframe()

In [8]:
pred.head(4)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even


In [28]:
newob = pred.loc[:0,'p0':'p63'].to_dict(orient='records')[0]
#newob

In [29]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [30]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[1.18610011e-09, 0.000214847227, 0.997517824, 6.16718637e-07, 3.61629e-14, 3.92662153e-10, 2.97701681e-05, 2.11727269e-13, 0.00223691668, 4.15889261e-08]], deployed_model_id='1563391185488183296', explanations=None)

In [31]:
prediction.predictions[0]

[1.18610011e-09,
 0.000214847227,
 0.997517824,
 6.16718637e-07,
 3.61629e-14,
 3.92662153e-10,
 2.97701681e-05,
 2.11727269e-13,
 0.00223691668,
 4.15889261e-08]

In [33]:
np.argmax(prediction.predictions[0])

2

### Get Predictions: REST

In [38]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [49]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      1.18610011e-09,
      0.000214847227,
      0.997517824,
      6.16718637e-07,
      3.61629e-14,
      3.92662153e-10,
      2.97701681e-05,
      2.11727269e-13,
      0.00223691668,
      4.15889261e-08
    ]
  ],
  "deployedModelId": "1563391185488183296"
}


### Get Predictions: gcloud (CLI)

In [50]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[1.18610011e-09, 0.000214847227, 0.997517824, 6.16718637e-07, 3.61629e-14, 3.92662153e-10, 2.97701681e-05, 2.11727269e-13, 0.00223691668, 4.15889261e-08]]


---
## Remove Resources
see notebook "XX - Cleanup"